In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request
from lxml import html
import requests
import re
from bs4 import element
import os

import time
from urllib.request import urlopen
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
from selenium.webdriver.common.keys import Keys

import pandas as pd
from IPython.display import display
from datetime import date 

In [2]:
os.getcwd()

'D:\\SMU\\Year4Sem1\\IS434\\IS434 project\\IS434_PJ\\WebScraper'

In [3]:
driver_option = webdriver.ChromeOptions()
# driver_option.add_argument('--headless')
# driver_option.add_argument('--incognito')
chromedriver_path = 'D://SMU//Year4Sem1//IS434//chromedriver_win32//chromedriver.exe' # Change this to your own chromedriver path!

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

In [4]:
# Get links for forum pages needed to be scraped

def get_links(browser):
    links = []
    link_soup = BeautifulSoup(browser.page_source, 'html.parser')
    link_code_soup = link_soup.find_all('div',attrs={'class':'w3i0q1-0 iZwAcY jdbj3f-0 fstzRi'})
    for article in link_code_soup:
        link = article.a["href"]
        links.append("https://seedly.sg" + link)
    return links

In [19]:
# Stores url links of each topic and saves them into a list

query_list = ['stocks-discussion','investments','insurance','property','credit-card','savings','shopping','lifestyle','family','retirement','career','cpf','loans','bank-account','money-fm-893-show','entrepreneurship','multi-currency-cards']

links = []

for item in query_list:
    url = 'https://seedly.sg/questions/topic/' + item

    browser = create_webdriver()
    browser.get(url)
    
    hasLoadMore = True
    while hasLoadMore:
        time.sleep(3)
        try:
            browser.find_element_by_css_selector('div.idq3j8-4.jOgrrt').click()
        except:
            hasLoadMore = False
            
    links.extend(get_links(browser))

In [5]:
# Save scraped data into a .csv file

def store_into_file(content_dict,counter,filename):
    content_df = pd.DataFrame.from_dict(content_dict,orient="index",columns=['content'])
    outfile = open("seedly-forum/" + filename + str(counter) + ".csv", 'w+', encoding="utf-8")
    content_df.to_csv(outfile)
    outfile.close()
    

In [6]:
# Retrieve the bs4 data of the url to be scraped.

def get_content(browser,counter):
    content_dict = {}
    link_soup = BeautifulSoup(browser.page_source, 'html.parser')
    counts = 0
    question = link_soup.find('div',attrs={'class':'w3i0q1-0 iZwAcY jdbj3f-0 fstzRi'})
    if not question:
        return
    content_dict["question"] = question.text
    asked_index = question.text.find("Asked")
    filename = ""
    if asked_index != -1:
        filename = question.text[:asked_index]
    size = len(link_soup.find_all('div',attrs={'class':'jdbj3f-0 fstzRi'}))
    for i,content in enumerate(link_soup.find_all('div',attrs={'class':'jdbj3f-0 fstzRi'})):
        if i+1 != size:
            content_dict[str(i)] = content.text
    store_into_file(content_dict,counter,filename)
    print(counter)

In [7]:
# Runs the links stored to get BS4 object to have data scraped

def run_link(links,browser):
    # Counter to indicate start line of crawling. Used to continue work should we not be able to crawl everything in a single run
    counter = 6399
    for link in links:
        browser.get(link)
        
        hasLoadMore = True
        while hasLoadMore:
            time.sleep(3)
            try:
                browser.find_element_by_css_selector('div.e4za3s-4 GBeDl').click()
            except:
                hasLoadMore = False
        print(link)
        get_content(browser,counter)
        counter+=1

In [20]:
links

['https://seedly.sg/questions/any-news-on-charisma-energy',
 'https://seedly.sg/questions/stocks-discussion-sgx-micro-mechanics-holdings-ltd-sgx-5dd',
 'https://seedly.sg/questions/how-should-i-start-investing-at-18yo-with-10k',
 'https://seedly.sg/questions/how-can-i-make-use-of-my-100-200-extra-every-month',
 'https://seedly.sg/questions/what-is-a-management-buyout-mbo',
 'https://seedly.sg/questions/is-it-better-to-buy-ut-than-etf-as-part-of-my-portfolio',
 'https://seedly.sg/questions/stock-discussion-singtel-sgx-z74-si',
 'https://seedly.sg/questions/how-do-you-convert-sgd-to-usd-to-invest-in-us-stock-markets-with-minimum-currency-loss',
 'https://seedly.sg/questions/how-do-you-balance-dca-vs-time-in-the-market-let-s-say-i-have-30k-now-ready-to-be-invested',
 'https://seedly.sg/questions/what-do-we-have-to-look-out-for-before-investing-in-ut',
 'https://seedly.sg/questions/how-do-people-estimate-how-much-to-invest-in-one-counter-and-also-per-transaction-do-you-set-aside-different-

In [29]:
browser = create_webdriver()
run_link(content_list,browser)

https://seedly.sg/questions/what-are-the-top-job-search-portals-or-websites-in-singapore
6399
https://seedly.sg/questions/since-income-tax-is-so-high-if-we-look-at-it-this-way-bonuses-just-cover-the-income-tax-doesn-t-it
6400
https://seedly.sg/questions/when-u-get-a-bonus-at-work-do-you-1-spend-it-on-yourself-family-or-2-invest-it
6401
https://seedly.sg/questions/i-want-to-start-my-own-youtube-channel-suppose-i-can-monetise-my-views-and-profit-from-affiliate-marketing-on-my-videos-and-social-media-accounts-do-i-need-to-declare-that-as-my-income-tax
6402
https://seedly.sg/questions/what-is-a-decent-amount-to-have-in-a-bank-at-around-19-years-old
6403
https://seedly.sg/questions/would-you-intern-at-a-new-small-startup-with-reputable-founders-and-investor-backing-but-they-are-literally-just-starting-your-department-and-haven-t-even-hired-your-direct-superior-yet
6404
https://seedly.sg/questions/what-are-some-good-networking-events-in-singapore-for-smes-small-businesses
6405
https://seedly

https://seedly.sg/questions/would-you-rather-make-30-000-a-year-doing-a-job-you-enjoy-or-making-100-000-doing-a-job-you-don-t-like
6453
https://seedly.sg/questions/nus-computer-science-versus-ntu-double-degree-in-business-and-computing-which-one-should-i-go-for
6454
https://seedly.sg/questions/lifebeyondgrades-share-your-psle-score-and-where-you-are-in-life-here-now
6455
https://seedly.sg/questions/how-do-i-become-a-portfolio-manager-in-a-hedge-fund
6456
https://seedly.sg/questions/giving-you-a-chance-again-back-to-age-20-30-40-what-busines-job-industry-you-like-to-venture-why
6457
https://seedly.sg/questions/how-long-would-you-stay-in-your-current-company-and-decide-it-is-time-to-move-on
6458
https://seedly.sg/questions/my-job-pays-well-but-the-management-is-extremely-unreasonable-should-i-start-looking-for-a-new-job
6459
https://seedly.sg/questions/should-i-switch-jobs-if-i-get-a-better-offer-even-though-i-am-confortable-in-my-current-one
6460
https://seedly.sg/questions/what-s-the-s

https://seedly.sg/questions/do-most-people-have-a-side-gig-on-top-of-their-normal-9-6-day-job
6510
https://seedly.sg/questions/i-am-thinking-of-taking-up-a-2-yr-part-time-masters-program-from-sim-global-warwick-in-particular-which-will-cost-32k-do-you-think-masters-are-worth-it-does-it-help-to-enhance-my-career-progression
6511
https://seedly.sg/questions/is-an-mba-worth-it-considering-the-time-and-money-involved
6512
https://seedly.sg/questions/what-is-the-financial-impact-of-a-person-who-serve-national-service-versus-someone-who-does-not
6513
https://seedly.sg/questions/what-are-the-additional-effective-sources-of-income-we-should-consider-grab-mlm-tuition
6514
https://seedly.sg/questions/is-fee-based-financial-advisory-a-good-career-if-i-wish-to-provide-unbiased-finance-advice-to-people
6515
https://seedly.sg/questions/noob-question-if-i-start-a-business-overseas-do-i-need-to-pay-my-taxes-and-cpf
6516
https://seedly.sg/questions/where-can-i-source-for-scholarships-to-apply-for-my-un

https://seedly.sg/questions/i-was-promoted-in-my-job-but-the-stress-and-additional-responsibilities-cause-me-to-not-enjoy-my-job-anymore-can-i-ask-for-a-demotion
6568
https://seedly.sg/questions/i-just-left-my-company-last-week-without-a-job-in-the-pipeline-and-i-received-a-job-offer-from-a-company-same-industry-but-it-comes-at-17-pay-cut-should-i-take-it-up
6569
https://seedly.sg/questions/could-i-ask-the-community-for-some-tips-on-how-to-do-up-a-good-resume-or-are-there-any-online-tools-to-do-so
6570
https://seedly.sg/questions/i-currently-work-at-an-sme-in-the-tech-sector-however-i-recently-was-given-an-offer-to-jump-to-an-mnc-where-i-will-be-managing-a-team-of-45-instead-of-12-how-do-i-prepare-myself-for-this
6571
https://seedly.sg/questions/how-to-get-job-in-singapore
6572
https://seedly.sg/questions/i-come-from-a-single-parent-home-and-i-don-t-want-my-mum-to-give-me-allowance-anymore-where-can-i-look-for-a-job-that-will-hire-a-16-years-old
6573
https://seedly.sg/questions/i-just-

https://seedly.sg/questions/how-to-become-a-millionaire
6619
https://seedly.sg/questions/what-do-the-community-think-about-having-a-long-term-endowment-plan-e-g-20-years-for-someone-who-just-started-working-what-are-the-pros-and-cons-thanks-in-advance
6620
https://seedly.sg/questions/do-i-need-a-licence-to-sell-home-made-goods-food-drinks-online-in-singapore
6621
https://seedly.sg/questions/is-it-allowed-if-for-us-nsfs-to-start-looking-for-jobs-8-months-before-ord
6622
https://seedly.sg/questions/anyone-knows-what-are-the-best-savings-account-for-entrepreneurs-since-i-don-t-have-a-credited-salary-which-is-one-of-the-main-requirements-of-such-byob-account
6623
https://seedly.sg/questions/what-do-you-look-for-in-a-job-besides-the-pay-what-is-important-to-you
6624
https://seedly.sg/questions/i-am-aspiring-to-be-an-entrepreneur-after-immediately-after-ns-what-are-some-resources-for-people-like-me-that-we-can-dig-on
6625
https://seedly.sg/questions/what-should-i-choose-a-good-boss-or-better

...

https://seedly.sg/questions/anyone-alarmed-by-ocbc-s-earnings
7471
https://seedly.sg/questions/where-can-i-park-my-money-which-gives-me-the-highest-form-of-return-with-high-liquidity-for-me-to-re-enter-the-market-when-it-is-back-in-the-downtrend
7472
https://seedly.sg/questions/hello-seedly-i-have-50k-that-i-m-taking-out-from-the-fd-today-15k-in-my-posb-savings-in-my-bank-another-10k-in-my-ocbc-365-the-acct-i-use-daily
7473
https://seedly.sg/questions/looking-for-a-priority-banking-card-with-useful-benefits-perks-any-recommendations
7474
https://seedly.sg/questions/does-it-make-sense-to-redeem-ssb-with-lower-step-up-interest-1-4x-2-0x-for-1st-3-4-yrs-but-with-attractive-step-up-in-last-few-yrs-arises-to-eg-2-11-2-31-mar-apr-18-tranche-average-interest
7475
https://seedly.sg/questions/which-saving-account-is-better-for-salary-credit-only
7476
https://seedly.sg/questions/how-do-i-indicate-bank-transfers-under-my-balance-account-as-part-of-my-expenditure-in-the-report-for-the-seedly-app
7

https://seedly.sg/questions/i-would-like-to-start-investing-and-have-created-a-cdp-account-and-all-but-am-not-really-sure-what-to-invest-in-how-do-i-really-start
7531
https://seedly.sg/questions/you-have-500000-suddenly-inherited-to-you-you-have-low-income-job-as-a-fresh-poly-graduate-you-have-no-insurance-yet-suddenly-you-had-ci-do-you-need-to-seek-msw-for-financial-assistance
7532
https://seedly.sg/questions/i-ve-just-started-a-sole-proprietorship-i-want-to-separate-business-profits-and-personal-savings-should-i-open-a-corporate-account-or-use-my-dbs-multi-currency-account-to-receive-all-revenues
7533
https://seedly.sg/questions/as-a-fresh-grad-what-is-some-of-the-best-insurance-plans-and-savings-accounts-to-consider
7534
https://seedly.sg/questions/why-do-we-buy-insurance-retirement-policy-when-the-cpf-provides-4-6-interest-returns-for-retirement-account
7535
https://seedly.sg/questions/i-am-25-this-year-and-new-to-the-workforce-my-parents-are-getting-old-and-i-read-about-creating-a

https://seedly.sg/questions/does-anyone-invest-in-posb-saver-any-thoughts
7580
https://seedly.sg/questions/should-i-leave-my-money-in-uk-bank-or-transfer-back-at-loss-to-singapore-to-invest
7581
https://seedly.sg/questions/where-to-park-50k-spare-cash
7582
https://seedly.sg/questions/should-i-save-with-dbs-multiplier-or-where-else-should-i-put-my-money-the-condition-of-maintaining-3000-per-month-is-that-the-net-balance-or-total-sum-of-transactions-in-out
7583
https://seedly.sg/questions/what-is-the-difference-between-rebate-cash-rebate-and-cash-back-in-credit-cards
7584
https://seedly.sg/questions/where-to-start-in-savings-budgeting-and-investment
7585
https://seedly.sg/questions/what-is-the-most-cost-effective-way-for-me-to-transfer-sgd-sing-dollar-to-rmb-chinese-yuan
7586
https://seedly.sg/questions/what-is-the-best-ewallet-app-there-are-too-many-around-and-i-m-confused
7587
https://seedly.sg/questions/will-phd-stipends-be-considered-as-salary-crediting-for-dbs-multiplier-account-or-

https://seedly.sg/questions/what-is-your-opinion-on-transferring-funds-from-one-s-ordinary-account-to-their-special-account-are-there-more-advantages-than-disadvantages-to-this-approach-and-what-are-they
7637
https://seedly.sg/questions/other-than-adding-cash-to-the-till-how-will-topping-up-one-s-spouse-s-cpf-account-benefit-non-working-spouses-with-low-cpf-balances
7638
https://seedly.sg/questions/you-stand-to-receive-a-tax-relief-of-up-to-14-000-from-topping-up-your-or-your-families-cpf-accounts-are-there-any-conditions-to-take-note-of-where-this-tax-relief-is-concerned
7639
https://seedly.sg/questions/what-are-the-most-common-misconceptions-that-singaporeans-have-about-cpf-that-prevents-them-from-maximising-their-savings-and-how-should-singaporeans-avoid-being-misinformed
7640
https://seedly.sg/questions/what-are-the-differences-between-the-various-electricity-retailers-like-keppel-electric-geneco-ohm-electricity-and-iswitch
7641
https://seedly.sg/questions/besides-cost-is-there-any

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=77.0.3865.120)


In [54]:
to_store = ""
for link in links:
    to_store += link + ","

NameError: name 'links' is not defined

In [36]:
# Stores links names to identify at which line should we continue off from where we were scraping

f= open("seedly-forum/" + "link-names" +".txt","w+", encoding="utf-8")
f.write(to_store)
f.close()

In [30]:
# Get all the links and store them as a string

f = open("seedly-forum/link-names.txt","r")
contents = ""
if f.mode == "r":
    contents = f.read()
    print(contents)

https://seedly.sg/questions/any-news-on-charisma-energy,https://seedly.sg/questions/stocks-discussion-sgx-micro-mechanics-holdings-ltd-sgx-5dd,https://seedly.sg/questions/how-should-i-start-investing-at-18yo-with-10k,https://seedly.sg/questions/how-can-i-make-use-of-my-100-200-extra-every-month,https://seedly.sg/questions/what-is-a-management-buyout-mbo,https://seedly.sg/questions/is-it-better-to-buy-ut-than-etf-as-part-of-my-portfolio,https://seedly.sg/questions/stock-discussion-singtel-sgx-z74-si,https://seedly.sg/questions/how-do-you-convert-sgd-to-usd-to-invest-in-us-stock-markets-with-minimum-currency-loss,https://seedly.sg/questions/how-do-you-balance-dca-vs-time-in-the-market-let-s-say-i-have-30k-now-ready-to-be-invested,https://seedly.sg/questions/what-do-we-have-to-look-out-for-before-investing-in-ut,https://seedly.sg/questions/how-do-people-estimate-how-much-to-invest-in-one-counter-and-also-per-transaction-do-you-set-aside-different-investment-amounts-for-each-category-like-

In [31]:
# Store each links as a string into content_list. Use index to indetify which url to continue to scrape from

content_list = contents.split(",")

In [19]:
stop_index = content_list.index("https://seedly.sg/questions/how-do-you-choose-the-right-banks-to-bank-with-deposits-insurance-investments-loans")

In [20]:
stop_index

4665

In [32]:
content_list = content_list[7689:]
content_list

['https://seedly.sg/questions/is-dbs-multi-currency-or-youtrip-better-as-a-multi-currency-wallet-for-foreign-transactions',
 '']

In [26]:
len(content_list) - 6399

1292

In [28]:
len(content_list)

1292